In [28]:
import os
import cv2
import numpy as np
import random
import sys
import testGMM

In [29]:
def trainGMM(K, img, max_iter):
    # user defined converge threshold
    converge_criteria = 0

    def initialize():
        random_w = random.randint(0,img.shape[0])
        random_h = random.randint(0,img.shape[1])
        mean = [[img[random_w][random_h][0]],[img[random_w][random_h][1]],[img[random_w][random_h][2]]]
        # generate a random positive-semidefinete matrix as covariance matrix
        A = np.random.randint(1,50,(3,3))
        cov = np.dot(A, A.transpose())
        scaling = random.randint(1,10)
        return [scaling,mean,cov]

    params = [initialize() for cluster in range(K)]
    # Structure of para:
    # [[scale,mean,covariance],[scale,mean,covariance],[scale,mean,covariance]...]
    # scale is a int. Mean is a 3x1 matrix. Covariance is a 3x3 matrix
    sum_means = 5
    sum_prev_means = 0
    iter = 0
    while iter <= max_iter and abs(sum_means - sum_prev_means) > converge_criteria:
        # TODO: define sum_means and sum_prev_means
        # Expectation step - assign points to clusters, get cluster weight
        weights = []
        for cluster in range(K):
            # weight for a single cluster
            cluster_weights = []
            cluster_scaling, cluster_mean, cluster_cov = params[cluster]
            for w in range(len(img[:, 0, 0])):
                for h in range(len(img[0, :, 0])):
                    pix = np.asmatrix([[img[w][h][0]], [img[w][h][1]], [img[w][h][2]]])
                    likelihood = testGMM.get_likelihood(pix, cluster_mean, cluster_cov)
                    denom = 0
                    ## calculate weight at position (w, h)
                    weight = cluster_scaling * likelihood
                    cluster_weights.append(weight) # probability of each pixel belonging to this cluster
            weights.append(cluster_weights) # weights for all clusters 1 to K,
            # weights[i][j]is the probability of the jth pixel belonging to the ith cluster


        # Maximization step - get new scaling, mean, and cov for each cluster
        for i in range(K):
            j = 0   # pixel index
            new_scaling = 0
            new_cov_num = []
            new_mean_nom = 0
            sum_cluster_weights = 0

            for w in range(len(img[:, 0, 0])):
                for h in range(len(img[0, :, 0])):
                    pix = np.asmatrix([[img[w][h][0]], [img[w][h][1]], [img[w][h][2]]])
                    new_mean_nom += weights[i][j] * pix
                    sum_cluster_weights += weights[i][j]
                    j += 1
            new_mean = new_mean_nom / sum_cluster_weights
            new_scaling = sum_cluster_weights / (w*h)

            for w in range(len(img[:, 0, 0])):
                for h in range(len(img[0, :, 0])):
                    pix = np.asmatrix([[img[w][h][0]], [img[w][h][1]], [img[w][h][2]]])
                    new_cov_num += weights[i][j] * (pix - new_mean).dot((pix - new_mean).T)
            new_cov = new_cov_num / sum_cluster_weights

            # update model
            params[i] = (new_scaling, new_mean, new_cov)

        i += 1





In [30]:
input_dir = "train_images"
for img_name in os.listdir(input_dir):
    img = cv2.imread(os.path.join(input_dir, img_name))
    trainGMM(5,img,200)
    break

OverflowError: math range error